## Multivaraite Datenanalyse
# Lineare Diskriminanzanalyse (LDA)
### Michael Araz, Daniel Hasenklever, Stefan Pede

In [ ]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

mmPfad = r"D:\C\Uni\Master\KSS\MV_Analyse\Messmatrix.csv"

### Eigenschaften der Messmatrix 

In [ ]:
df = pd.read_csv(mmPfad)
print("Anzahl der Kennwerte: "+str(df.shape[1]))
print("Anzahl der vermessenen Rohre: "+str(df.shape[0]))
print("Anzahl der gefahrenen Produkte: "+str(df.groupby(["Header_Leitguete","Header_Soll_AD","Header_Soll_WD"])["Header_Pseudonummer"].agg(["count"]).shape[0]))
print("Anzahl der Walzlose: "+str(len(pd.unique(df["Header_Walzlos"]))))
print("\nAuszug:")
df.head()

### Produkte

In [ ]:
df.groupby(["Header_Leitguete","Header_Soll_AD","Header_Soll_WD"])["Header_Pseudonummer"].agg(["count"])

### Vorverarbeiten der Kennwerte
Entfernen derjenigen Spalten und Zeilen mit zu vielen ungültigen/fehlenden Einträgen

In [ ]:
def processNaNs(df, bounds):
## df: pandas dataframe that is to be processed
## bounds: list of percentages for column and row filtering, e.g. [15, 5]
    relNaNsCol = np.sum(np.isnan(df))/df.shape[0]*100
    # schmeiße zunächst alle Spalten heraus, die mehr als bestimmte Prozent an NaNs haben
    spaltenSchranke = bounds[0] # % der NaNs in Spalte
    keep = [i for i in np.arange(len(relNaNsCol)) if relNaNsCol[i] <= spaltenSchranke]
    dfVV = df[df.columns[keep]] # extrahiere Spalten

    # gleiches auf Zeilen anwenden
    zeilenSchranke = bounds[1] # % der NaNs in Zeile
    relNaNsRow = dfVV.isnull().sum(axis=1)/dfVV.shape[1]*100
    keep = [i for i in np.arange(len(relNaNsRow)) if relNaNsRow[i] <= zeilenSchranke]
    dfVV2 = dfVV.iloc[keep] #extraheire Zeilen

    #übrige NaNs mit Mittelwert aus Spalten auffüllen
    dfVV2 = dfVV2.fillna(dfVV2.mean())

    # Ausgabe
    print("Daten nach Vorverarbeitung:")
    print("Anzahl der Kennwerte: "+str(dfVV2.shape[1]))
    print("Anzahl der vermessenen Rohre: "+str(dfVV2.shape[0]))
    print("Anzahl der gefahrenen Produkte: "+str(dfVV2.groupby(["Header_Leitguete","Header_Soll_AD","Header_Soll_WD"])["Header_Pseudonummer"].agg(["count"]).shape[0]))
    print("Anzahl der Walzlose: "+str(len(pd.unique(dfVV2["Header_Walzlos"]))))

    return dfVV2

In [ ]:
def labelData(pdDaFr, label):
# Diese Funktion erhält als Eingabegrößen das vorverarbeitete Dataframe und den Namen der Header-Spalte, die als Label
# dienen soll, mittelwertfreie Rückgabe ohne die Header-Informationen
    pdDaFr = pdDaFr.copy()
    anzHeaderSpalten = 6
    preLabels = pd.unique(pdDaFr[label])
    postLabels = np.arange(len(preLabels))

    # kopiere Spalte mit Labels
    class_label = pdDaFr[label].values
    # kopiere nicht den Header
    pdDaFr = pdDaFr[pdDaFr.columns[6:]]
    pdDaFr = pdDaFr-pdDaFr.mean()
    # ersetze die Labels durch Integer-Labels
    for i in postLabels:
        class_label[class_label == preLabels[i]] = postLabels[i]
    #passe Datentyp an
    class_label = class_label.astype(np.int64)
    
    return(pdDaFr, class_label)

In [ ]:
# preprocess data and label it
dfVV2 = processNaNs(df, [15,5])
label = "Header_Walzlos" #als Beispiel
data_preProc_LDA, labels = labelData(dfVV2, label)

In [ ]:
def getTrainingAndTestData(data, labels, fraction, minNoTubesPerSet):
## split data into training- and test-data
## data: pandas dataframe that is to be splitted
## labels: label for each row of the dataframe
## fraction: e.g. 5 to get a fifth, i.e. 20%, of the original data as test data
## minNoTubesPerSet: minimum number of tubes in a set to consider it, e.g. 20

    training_set = pd.DataFrame()
    test_set = pd.DataFrame()
    training_labels=np.array([])
    test_labels = np.array([])
    
    for i in np.unique(labels):
        anz_Rohr_in_Walzlos = np.sum(labels==i)
        if anz_Rohr_in_Walzlos > minNoTubesPerSet:
            test_data_size = int(anz_Rohr_in_Walzlos/fraction)
            training_data_size = anz_Rohr_in_Walzlos-test_data_size
            
            tmp=data[labels==i]
            training_set=training_set.append(tmp[:training_data_size])
            test_set = test_set.append(tmp[training_data_size:])

            training_labels = np.concatenate((training_labels,i*np.ones(training_data_size)))
            test_labels = np.concatenate((test_labels,i*np.ones(test_data_size)))
    
    return ([training_set, test_set],[training_labels, test_labels])

In [ ]:
# split data into test and training data
sets, sets_labels = getTrainingAndTestData(data_preProc_LDA, labels, 5, 30)
training_set = sets[0]
test_set = sets[1]
training_labels = sets_labels[0]
test_labels = sets_labels[1]

In [ ]:
def dropCorrelatedColumns(sets, covBound):
    from scipy import stats

    ## suche hohe Werte in Kovarianz-Matrix
    if covBound < 0:
        text = "korrelieren negativ"
        kovBool = pd.DataFrame(np.cov((stats.zscore(sets[0]).T)) < covBound)
    else:
        text = "korrelieren positiv"
        kovBool = pd.DataFrame(np.cov((stats.zscore(sets[0]).T)) > covBound)        
    ## suche diejenigen, die nicht auf Diagonale liegen
    korr = []
    for a,b in zip(np.where(kovBool)[0], np.where(kovBool)[1]):
        if (a != b):
            korr.append([a,b])
    print(korr)
    ## sortiere diese und finde einzigartige
    korr = [sorted(i) for i in korr]
    korrWD = []
    for i in korr:
        if i not in korrWD:
            korrWD.append(i)
            print(sets[0].columns[i[0]]," und ",sets[1].columns[i[1]],text)
    ## erhalte Indizes der korrelierenden Spalten
    drop = []
    for i in korrWD:
        drop.append(i[1])

    ## lösche diese aus den vorliegenden Sets           
    retTrainingSet = sets[0].drop(sets[0].columns[drop], axis=1)
    retTestSet = sets[1].drop(sets[1].columns[drop], axis=1)
    return(retTrainingSet, retTestSet)

In [ ]:
## drop correlated columns
## LDA zeigt Warnung, dass Variablen korrelieren bis zu einer Kovarianz von 0.9 an
training_set_noCorr, test_set_noCorr = dropCorrelatedColumns([training_set, test_set], 0.9)

## LDA

Reduktion der Daten um korrelierende Eingangsgrößen

In [ ]:
# train classificator
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
sklearn_lda = LDA(n_components=10)
data_lda = sklearn_lda.fit_transform(training_set_noCorr, training_labels)

In [ ]:
# test
# with test-data
prediction = sklearn_lda.predict(test_set_noCorr)
print("Test-Set-Prediction: ",np.mean(prediction == test_labels))
#with training-data
pred = sklearn_lda.predict(training_set_noCorr)
print("Training-Set-Prediction: ",np.mean(pred == training_labels))

In [ ]:
## STRANGE
## die Genauigkeit des Klassifikators ändert sich nicht für verschiedene n_components
## sklearn_lda.coef_ enthält 2D statt 1D-Daten
## kann es sein, dass für jedes Walzlos ein eigener Klassifikator erstellt wird oder so ähnlich?